In [1]:
from mds.database.mongo import GetConfig
from mds.database.minio import GetMinioConfig
from mds.database.config import MONGO_DATABASE, MONGO_COLLECTION
from mds.models.download import Download
from mds.models.dataset import Dataset
from mds.models.project import Project
from mds.models.organization import Organization
from mds.models.user import User

In [2]:
mongo_client = GetConfig()
mongo_db = mongo_client[MONGO_DATABASE]
mongo_collection = mongo_db[MONGO_COLLECTION]

In [3]:
minio_client = GetMinioConfig()

In [14]:
# create a user
user1_data = {
    "@id": "ark:99999/test-user1",
    "name": "Test User1",
    "type": "Person",
    "email": "testuser1@example.org",
    "password": "test1",
    "organizations": [],
    "projects": [],
    "datasets": [],
    "software": [],
    "computations": [],
    "evidencegraphs": []
}

user1 = User(**user1_data)

In [15]:
print(user1.create(mongo_collection))

Success: True	Message: 	StatusCode: 200


In [16]:
# create an org
org_data = {
    "@id": "ark:99999/test-org",
    "name": "test organization",
    "@type": "Organization",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    }
}

org = Organization(**org_data)

In [17]:
print(org.create(mongo_collection))

Success: True	Message: 	StatusCode: 201


In [18]:
# create a project
project_data = {
    "@id": "ark:99999/test-org/test-proj",
    "name": "Test Project",
    "@type": "Project",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    },
    "memberOf": {
        "@id": "ark:99999/test-org",
        "name": "test organization",
        "@type": "Organization",
    }
}

proj = Project(**project_data)


In [19]:
print(proj.create(mongo_collection))

Success: True	Message: 	StatusCode: 201


In [20]:
# create a dataset
dataset_data = {
    "@id": "ark:99999/test-org/test-proj/test-data",
    "@type": "Dataset",
    "name": "Test Dataset",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    }
}

ds = Dataset(**dataset_data)

In [21]:
print(ds.create(mongo_client))

ark:99999/test-user1
Success: True	Message: 	StatusCode: 201


In [12]:
# create a download
download_data = {
    "@id": "ark:99999/test-org/test-proj/test-data/file1.txt",
    "@type": "DataDownload",
    "name": "Test CSV",
    "encodingFormat": ".csv",
    "encodesCreativeWork": {
        "@id": "ark:99999/test-org/test-proj/test-data",
        "@type": "Dataset",
        "name": "Test Dataset",
    }
}

dload = Download(**download_data)

In [13]:
data_file = open("tests/test-data.csv", "rb")
print(dload.register(mongo_collection, minio_client, data_file))

Success: True	Message: 	StatusCode: 201


## Testing with API

In [27]:
import requests
import json
from requests_toolbelt.multipart.encoder import MultipartEncoder

root_url = "http://localhost:8000/"

In [23]:
# create user
user_create = requests.post(root_url + "user", json=user1_data)

In [ ]:
print(user_create.status)

In [25]:
# create organization
organization_create = requests.post(root_url + "organization", json=org_data)

In [ ]:
dataset_create = requests.post(root_url + f"dataset/", data=json.dumps(dataset_data))
dataset_create.json()

In [32]:
download_data = {
    "@id": "ark:99999/test-org/test-proj/test-data/file1.txt",
    "@type": "DataDownload",
    "name": "Test CSV",
    "encodingFormat": ".csv",
    "encodesCreativeWork": {
        "@id": "ark:99999/test-org/test-proj/test-data",
        "@type": "Dataset",
        "name": "Test Dataset",
    }
}


mp_encoder = MultipartEncoder(
    fields={
        'download': json.dumps(download_data),
        # plain file object, no filename or mime type produces a
        # Content-Disposition header with just the part name
        'file': ('test-data', open('tests/test-data.csv', 'rb'), 'text/plain'),
    }
)

# upload a software to minio object store
# dataset_upload = {'download': str(download_data), 'file': open("tests/test-data.csv", "rb")}

data_download_upload = requests.post(root_url + f"register",                              
    data=mp_encoder,  # The MultipartEncoder is posted as data, don't use files=...!
    # The MultipartEncoder provides the content-type header with the boundary:
    headers={'Content-Type': mp_encoder.content_type}
                )
                    
data_download_upload.json()

{'success': True, 'message': ''}

In [34]:
data_download_upload.request.body

b'--6df09a0e6e892513f974cec4ae2a8b67\r\nContent-Disposition: form-data; name="download"; filename="download"\r\n\r\n{\'@id\': \'ark:99999/test-org/test-proj/test-data/file1.txt\', \'@type\': \'DataDownload\', \'name\': \'Test CSV\', \'encodingFormat\': \'.csv\', \'encodesCreativeWork\': {\'@id\': \'ark:99999/test-org/test-proj/test-data\', \'@type\': \'Dataset\', \'name\': \'Test Dataset\'}}\r\n--6df09a0e6e892513f974cec4ae2a8b67\r\nContent-Disposition: form-data; name="file"; filename="test-data.csv"\r\n\r\nx, y, z\r\n1, 5, 10\r\n2, 10, 20\r\n3, 15, 30\r\n--6df09a0e6e892513f974cec4ae2a8b67--\r\n'

In [4]:
from fastapi import APIRouter, Response, UploadFile, Form, File


In [5]:
help(Form)

Help on function Form in module fastapi.param_functions:

Form(default: Any, *, media_type: str = 'application/x-www-form-urlencoded', alias: Optional[str] = None, title: Optional[str] = None, description: Optional[str] = None, gt: Optional[float] = None, ge: Optional[float] = None, lt: Optional[float] = None, le: Optional[float] = None, min_length: Optional[int] = None, max_length: Optional[int] = None, regex: Optional[str] = None, example: Any = PydanticUndefined, examples: Optional[Dict[str, Any]] = None, **extra: Any) -> Any

